In [26]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [27]:
load_dotenv()

True

In [28]:
apikey = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=apikey
)
print(client)

In [48]:
def stream_chatbot(message):
    chat_completion_stream = client.chat.completions.create(
        messages=[
            {"role":"user",
             "content":message,
             },
        ],
        model="gpt-3.5-turbo",
        stream=True
    )
    for chunk in chat_completion_stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")

In [49]:
def prompt_user():
    user_input = input("You: ")
    return user_input

In [50]:
def print_user_message(user_message):
    print("You: ", user_message)
    

In [51]:
def chatbot(user_message):
    print("Chatbot: ")
    stream_chatbot(user_message)

In [52]:
prompt = prompt_user()
print_user_message(prompt)
chatbot(prompt)

You:  Hiiii, it's me. Akuske!
Chatbot: 
Hello Akuske! How can I assist you today?